In [1]:
#Importing Important Libraries
import requests
from bs4 import BeautifulSoup

link = 'https://books.toscrape.com/catalogue/page-1.html'

#Sending a request to the website(link)
res = requests.get(link)

#Creating a soup using BeautifulSoup
soup = BeautifulSoup(res.text,'html.parser')

In [2]:
# code
book = soup.find_all('li',class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3')

data=[]
for sp in soup.find_all('li',class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):
    #print(sp)
	#Finding different aspects to be scraped from the first page 
    book_link   = "https://books.toscrape.com/catalogue/" + sp.find_all('a')[-1].get('href')
    
    title       = sp.find_all('a')[-1].get('title')
    
    img_link    = "https://books.toscrape.com/" + sp.find('img').get('src')[3:]
    
    book_rating = (sp.find('p').get('class')[-1])
    
    price       = sp.find('p',class_='price_color').text[1:]
    
    stock       =  sp.find('p',class_ = 'instock availability').text.strip()
	#Appending all the data into a list(data)
    data.append([title,book_rating,price,stock,book_link,img_link])

In [3]:
from tqdm import tqdm
Multiple_Pages = []
#tqdm used for better representation.
for page in tqdm(range(1,51)):
  	#using a for loop as there are 50 pages then creating a link using page-1,page-2...page-50
    link = 'https://books.toscrape.com/catalogue/page-'+str(page)+'.html'
    res = requests.get(link)
    soap = BeautifulSoup(res.text,'html.parser')
    #same code as in scraping for 1 page
    for sp in soap.find_all('li',class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):
       
        book_link   = "https://books.toscrape.com/catalogue/" + sp.find_all('a')[-1].get('href')

        title       = sp.find_all('a')[-1].get('title')

        img_link    = "https://books.toscrape.com/" + sp.find('img').get('src')[3:]

        book_rating = (sp.find('p').get('class')[-1])

        price       = sp.find('p',class_='price_color').text[1:]

        stock       =  sp.find('p',class_ = 'instock availability').text.strip()

        Multiple_Pages.append([title,book_rating,price,stock,book_link,img_link])

100%|██████████| 50/50 [01:20<00:00,  1.60s/it]


In [4]:
#Creating a Dataframe
import pandas as pd
Multiple_Pages_df = pd.DataFrame(data=Multiple_Pages)
Multiple_Pages_df = Multiple_Pages_df.rename(columns={0: 'Title', 1: 'Rating',2:'Price',3:'Stock Available',4:'Book Link',5:'Image Link'})
df_1 = Multiple_Pages_df

In [5]:
Multiple_Pages_df.to_csv('All Books.csv',index=False)

In [6]:
data_2 = []
for links in tqdm(Multiple_Pages_df['Book Link']):
    res        = requests.get(links)
    soup       = BeautifulSoup(res.text,'html.parser')
    
    Book_Genre       = soup.find('ul',class_ = 'breadcrumb').find_all('a')[2].text
    
    temp       = soup.find('table',class_ = 'table table-striped').find_all('td')    
    
    UPC              = temp[0].text
    price_exc_tax    = temp[2].text[1:]
    price_inc_tax    = temp[3].text[2:]
    tax              = temp[4].text[2:]
    availability     = temp[5].text
    reviews          = temp[6].text   
    data_2.append([Book_Genre,price_exc_tax,price_inc_tax,tax,UPC,availability,reviews])

100%|██████████| 1000/1000 [22:13<00:00,  1.33s/it] 


In [7]:
#Creating a dataframe
df_2 = pd.DataFrame(data_2)
df_2 = df_2.rename(columns={0:'Genre',1:'Price',2:"Price(Excluding Tax)",3:"Price(Including Tax)" ,4:"UPC",5:"Stock",6:"Reviews"})

#Saving Dataframe
df_2.to_csv('All Page.csv',index=False)

In [9]:
df = pd.DataFrame()
df['UPC']   = df_2['UPC']
df['Title'] = df_1['Title']
df['Genre']  = df_2['Genre']
df['Rating'] = df_1['Rating']
df['Price']   = df_2['Price']

df['Stock'] = df_2['Stock']
df['Review'] = df_2['Reviews']
df['Book Link'] = df_1['Book Link']
df['Cover Image'] = df_1['Image Link']

#Saving final scrap
df.to_csv("Final Scrap.csv")